**Dự án này được nấu bởi [Phạm Huỳnh Anh](https://github.com/PhamHuynhAnh16)**

# **Vui lòng không sử dụng dự án với bất kỳ mục đích nào vi phạm đạo đức, pháp luật, hoặc gây tổn hại đến cá nhân, tổ chức...**

# **Trong trường hợp người sử dụng không tuân thủ các điều khoản hoặc vi phạm, tôi sẽ không chịu trách nhiệm về bất kỳ khiếu nại, thiệt hại, hay trách nhiệm pháp lý nào, dù là trong hợp đồng, do sơ suất, hay các lý do khác, phát sinh từ, ngoài, hoặc liên quan đến phần mềm, việc sử dụng phần mềm hoặc các giao dịch khác liên quan đến phần mềm.**

In [ ]:
#@title **🌏 Cài đặt**
import os
import codecs
import tarfile
import urllib.request

from threading import Thread
from ipywidgets import Button
from IPython.display import clear_output

def download_library_1():
  with urllib.request.urlopen(codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/raivebazragf/hfe_1.gne.tm", "rot13")) as response:
    with tarfile.open(fileobj=response, mode="r:gz") as tar:
      tar.extractall(path="/usr/local/")

def download_library_2():
  with urllib.request.urlopen(codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/raivebazragf/hfe_2.gne.tm", "rot13")) as response:
    with tarfile.open(fileobj=response, mode="r:gz") as tar:
      tar.extractall(path="/usr/local/lib/python3.11")

print("👩🏻‍💻 Cài đặt...")

download_rvc = Thread(target=lambda: os.system("git clone https://github.com/PhamHuynhAnh16/Vietnamese-RVC /content/Vietnamese_RVC"))
download_rvc.start()

delete_a = Thread(target=lambda: os.system('rm -rf /usr/local/bin'))
delete_b = Thread(target=lambda: os.system('rm -rf /usr/local/lib/python3.11'))

delete_a.start()
delete_b.start()

delete_a.join()
delete_b.join()

download_a = Thread(target=download_library_1)
download_b = Thread(target=download_library_2)

download_a.start()
download_b.start()

download_a.join()
download_b.join()

#@markdown **💻 Cài đặt sẽ mất khoảng 2 phút để hoàn tất!**

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **📱 Mở giao diện sử dụng**
import os
import shutil

%cd /content/Vietnamese_RVC
#@markdown **Để trải nghiệm hết tính năng hãy dùng giao diện:) Còn muốn đơn giản thì không dùng giao diện**

#@markdown **Nếu biết có thể sử dụng biểu đồ để kiểm tra huấn luyện quá sức 👍**
su_dung_bieu_do = False #@param {type:"boolean"}

if su_dung_bieu_do:
  %load_ext tensorboard
  %tensorboard --logdir ./assets/logs --port=6870

!python main/app/app.py --share

# **Tùy chỉnh thêm 🧰**

In [ ]:
#@title **Kết nối hoặc ngắt kết nối với drive ☁**
import os
from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output, display


if os.path.exists("/content/drive"):
  print("🔗 Ngắt kết nối với drive...")
  try:
    drive.flush_and_unmount()
    clear_output()
    display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
  except Exception as e:
    raise ValueError(f'Đã xảy ra lỗi trong quá trình ngắt kết nối drive: {e}')
else:
  print('🔗 Kết nối với drive...')
  try:
    drive.mount('/content/drive')
    clear_output()
    display(Button(description="\u2714 Hoàn Tất!", button_style="success"))
  except Exception as e:
    raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

In [ ]:
#@title **Khởi động hoặc ngừng sao lưu 🛠**
import os
import time
import threading
import subprocess

from ipywidgets import Button
from google.colab import drive
from IPython.display import clear_output

logs_folder, weights_folder, audios_folder = '/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Nếu không tích vào ô nào thì sẽ ngừng kết nối ở phần đó**
khoi_dong_sao_luu_mo_hinh = False #@param {"type":"boolean"}
khoi_dong_sao_luu_am_thanh = False #@param {"type":"boolean"}
#@markdown **Đồng bộ là sẽ đồng bộ các thư mục sao lưu lại, việc thư mục bị xóa mất 1 tệp thì ở thư mục sao lưu tệp đó cũng sẽ bị xóa**
dong_bo_thu_muc = False #@param {"type":"boolean"}

%cd /content/Vietnamese_RVC

class Channel:
    def __init__(self, source, destination, sync_deletions=False, every=60, exclude = None):
        self.source = source
        self.destination = destination
        self.event = threading.Event()
        self.syncing_thread = threading.Thread(target=self._sync, args=())
        self.sync_deletions = sync_deletions
        self.every = every

        if not exclude: exclude = []
        if isinstance(exclude, str): exclude = [exclude]

        self.exclude = exclude
        self.command = ['rsync', '-aP']

    def alive(self):
        if self.syncing_thread.is_alive(): return True
        else: return False

    def _sync(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')

        while not self.event.is_set():
            subprocess.run(command)
            time.sleep(self.every)

    def copy(self):
        command = self.command

        for exclusion in self.exclude:
            command.append(f'--exclude={exclusion}')

        command.extend([f'{self.source}/', f'{self.destination}/'])

        if self.sync_deletions: command.append('--delete')
        subprocess.run(command)

        return True

    def start(self):
        if self.syncing_thread.is_alive():
            self.event.set()
            self.syncing_thread.join()

        if self.event.is_set(): self.event.clear()
        if self.syncing_thread._started.is_set(): self.syncing_thread = threading.Thread(target=self._sync, args=())

        self.syncing_thread.start()
        return self.alive()

    def stop(self):
        if self.alive():
            self.event.set()
            self.syncing_thread.join()

            while self.alive():
                if not self.alive(): break

        return not self.alive()

if not "logs_backup" in locals(): logs_backup = Channel("/content/Vietnamese_RVC/assets/logs", logs_folder, sync_deletions=dong_bo_thu_muc, every=40, exclude="mute")
if not "weights_backup" in locals(): weights_backup = Channel("/content/Vietnamese_RVC/assets/weights", weights_folder, sync_deletions=dong_bo_thu_muc, every=40)
if not "audio_backup" in locals(): audio_backup = Channel("/content/Vietnamese_RVC/audios", audios_folder, sync_deletions=dong_bo_thu_muc, every=40)

logs_backup.stop(); weights_backup.stop(); audio_backup.stop()

if os.path.exists('/content/drive/MyDrive'):
  if khoi_dong_sao_luu_mo_hinh:
    if not os.path.exists(logs_folder): os.makedirs(logs_folder)
    if not os.path.exists(weights_folder): os.makedirs(weights_folder)

    logs_backup.start(); weights_backup.start()
  else: logs_backup.stop(); weights_backup.stop()

  if khoi_dong_sao_luu_am_thanh:
    if not os.path.exists(audios_folder): os.makedirs(audios_folder)
    audio_backup.start()
  else: audio_backup.stop()
else:
  try:
    drive.mount('/content/drive')
  except Exception as e:
      raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

  if khoi_dong_sao_luu_mo_hinh:
    if not os.path.exists(logs_folder): os.makedirs(logs_folder)
    if not os.path.exists(weights_folder): os.makedirs(weights_folder)

    logs_backup.start(); weights_backup.start()
  else: logs_backup.stop(); weights_backup.stop()

  if khoi_dong_sao_luu_am_thanh:
    if not os.path.exists(audios_folder): os.makedirs(audios_folder)
    audio_backup.start()
  else: audio_backup.stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **♻️ Khởi động dọn rác**
import os
import time
import threading

from ipywidgets import Button
from google.colab import auth, drive
from IPython.display import clear_output

try:
  from googleapiclient.discovery import build
except:
  os.system("pip install google-api-python-client")
  from googleapiclient.discovery import build

class Clean:
    def __init__(self, every=60):
        self.service = build('drive', 'v3')
        self.every = every
        self.trash_cleanup_thread = None

    def delete(self):
        page_token = None

        while 1:
            response = self.service.files().list(q="trashed=true", spaces='drive', fields="nextPageToken, files(id, name)", pageToken=page_token).execute()

            for file in response.get('files', []):
                if file['name'].startswith("G_") and file['name'].endswith(".pth") or file['name'].startswith("D_") and file['name'].endswith(".pth"):
                    try:
                        self.service.files().delete(fileId=file['id']).execute()
                    except Exception as e:
                        raise RuntimeError(e)

            page_token = response.get('nextPageToken', None)
            if page_token is None: break

    def clean(self):
        while 1:
            self.delete()
            time.sleep(self.every)

    def start(self):
        self.trash_cleanup_thread = threading.Thread(target=self.clean)
        self.trash_cleanup_thread.daemon = True
        self.trash_cleanup_thread.start()

    def stop(self):
        if self.trash_cleanup_thread: self.trash_cleanup_thread.join()

%cd /content/Vietnamese_RVC

#@markdown **Sử dụng khi huấn luyện sẽ dọn bớt các tệp tin D, G trong thùng rác google drive**
khoi_dong_don_rac = False #@param {"type":"boolean"}

if khoi_dong_don_rac:
  if os.path.exists('/content/drive/MyDrive'):
    auth.authenticate_user()
  else:
    try:
      drive.mount('/content/drive')
    except Exception as e:
        raise ValueError(f'Đã xảy ra lỗi trong quá trình kết nối drive: {e}')

    auth.authenticate_user()
    Clean(every=40).start()
else: Clean().stop()

clear_output()
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **Tải dữ liệu sao lưu từ drive 📂**
import os
import shutil
from ipywidgets import Button
from IPython.display import clear_output, display

logs_folder, weights_folder, audios_folder ='/content/drive/MyDrive/model/logs', '/content/drive/MyDrive/model/weights', '/content/drive/MyDrive/audios'

#@markdown **Tải các mô hình huấn luyện để tiếp tục huấn luyện**
tai_mo_hinh = False # @param {"type":"boolean"}
#@markdown **Tải các âm thanh được sao lưu để tiếp tục sử dụng**
tai_am_thanh = False # @param {"type":"boolean"}

if os.path.exists("/content/drive/MyDrive"):
  if tai_mo_hinh:
    if len(os.listdir(logs_folder)) < 1 or len(os.listdir(weights_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/model"):
        shutil.copytree(logs_folder, "/content/Vietnamese_RVC/assets/logs", dirs_exist_ok=True)
        shutil.copytree(weights_folder, "/content/Vietnamese_RVC/assets/weights", dirs_exist_ok=True)

        clear_output()
      else: print("Không tìm thấy dữ liệu mô hình")
  elif tai_am_thanh:
    if len(os.listdir(audios_folder)) < 1: print("Không tìm thấy dữ Liệu")
    else:
      if os.path.exists("/content/drive/MyDrive/audios"):
        shutil.copytree(audios_folder, "/content/Vietnamese_RVC/audios", dirs_exist_ok=True)
        clear_output()
      else: print("Không tìm thấy dữ liệu âm thanh")
else: print("Google drive không được kết nối")

display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title **Dung hợp mô hình🌀**
import os
import torch

from ipywidgets import Button
from google.colab import files
from collections import OrderedDict
from IPython.display import clear_output, display

def fushion_model(name, pth_1, pth_2, ratio):
    if not name: raise ValueError("Hãy cung cấp tên mô hình!")

    if not name.endswith(".pth"): name = name + ".pth"

    if not pth_1 or not os.path.exists(pth_1) or not pth_1.endswith(".pth"): raise FileExistsError("Vui lòng cung cấp mô hình 1 hợp lệ")
    if not pth_2 or not os.path.exists(pth_2) or not pth_1.endswith(".pth"): raise FileExistsError("Vui lòng cung cấp mô hình 2 hợp lệ")

    def extract(ckpt):
        a = ckpt["model"]
        opt = OrderedDict()
        opt["weight"] = {}

        for key in a.keys():
            if "enc_q" in key: continue

            opt["weight"][key] = a[key]

        return opt

    try:
        ckpt1 = torch.load(pth_1, map_location="cpu")
        ckpt2 = torch.load(pth_2, map_location="cpu")

        if ckpt1["sr"] != ckpt2["sr"]: raise ValueError("Tốc độ lấy mẫu của 2 mô hình không giống nhau")

        cfg = ckpt1["config"]
        cfg_f0 = ckpt1["f0"]
        cfg_version = ckpt1["version"]
        cfg_sr = ckpt1["sr"]

        ckpt1 = extract(ckpt1) if "model" in ckpt1 else ckpt1["weight"]
        ckpt2 = extract(ckpt2) if "model" in ckpt2 else ckpt2["weight"]

        if sorted(list(ckpt1.keys())) != sorted(list(ckpt2.keys())): raise ValueError("Cấu trúc mô hình không giống nhau")

        opt = OrderedDict()
        opt["weight"] = {}

        for key in ckpt1.keys():
            if key == "emb_g.weight" and ckpt1[key].shape != ckpt2[key].shape:
                min_shape0 = min(ckpt1[key].shape[0], ckpt2[key].shape[0])
                opt["weight"][key] = (ratio * (ckpt1[key][:min_shape0].float()) + (1 - ratio) * (ckpt2[key][:min_shape0].float())).half()
            else: opt["weight"][key] = (ratio * (ckpt1[key].float()) + (1 - ratio) * (ckpt2[key].float())).half()

        opt["config"] = cfg
        opt["sr"] = cfg_sr
        opt["f0"] = cfg_f0
        opt["version"] = cfg_version
        opt["infos"] = "BRUH"

        output_model = os.path.join("assets", "weights")
        if not os.path.exists(output_model): os.makedirs(output_model, exist_ok=True)

        torch.save(opt, os.path.join(output_model, name))
    except Exception as e:
        raise Exception(f"Đã xảy ra lỗi: {e}")

def upload_model_files():
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, 'wb') as f:
            f.write(data)
        print(f'Tải Lên {name}')

    return uploaded

#@markdown **Tên của tệp mô hình khi lưu ra**
ten_mo_hinh = "" # @param {"type":"string","placeholder":"Tên mô hình"}
#@markdown **Đường dẫn của 2 mô hình cần dung hợp**
tep_mo_hinh_1 = "" # @param {"type":"string","placeholder":"assets\\\\weights\\\\My_Model_1.pth"}
tep_mo_hinh_2 = "" # @param {"type":"string","placeholder":"assets\\\\weights\\\\My_Model_2.pth"}
#@markdown **Chỉnh hướng về bên nào sẽ làm cho mô hình giống với bên đó**
ty_le_mo_hinh = 0.5 #@param {type:"slider", min:0, max:1, step:0.1}

%cd /content/Vietnamese_RVC

if not tep_mo_hinh_1:
    print("Hãy tải lên mô hình thứ 1:")
    uploaded_files_a = upload_model_files()
    duong_dan_mo_hinh_1 = list(uploaded_files_a.keys())[0]
elif not tep_mo_hinh_2:
    print("Hãy tải lên mô hình thứ 2:")
    uploaded_files_b = upload_model_files()
    duong_dan_mo_hinh_2 = list(uploaded_files_b.keys())[0]
else:
    duong_dan_mo_hinh_1 = os.path.join(tep_mo_hinh_1)
    duong_dan_mo_hinh_2 = os.path.join(tep_mo_hinh_2)

fushion_model(ten_mo_hinh, duong_dan_mo_hinh_1, duong_dan_mo_hinh_2, ty_le_mo_hinh)

clear_output()
display(Button(description="\u2714 Hoàn Tất", button_style="success"))

In [ ]:
#@title **Đọc thông tin mô hình📰**
import os
import torch

from datetime import datetime
from google.colab import files

duong_dan_mo_hinh = "" # @param {"type":"string","placeholder":"assets\\\\weights\\\\My-Model.pth"}

def model_info(path):
    if not path or not os.path.exists(path): raise ValueError("Không tìm thấy mô hình!")

    def prettify_date(date_str):
        if date_str == "Không tìm thấy thời gian tạo": return None

        try:
            return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f").strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
            return "Định dạng không hợp lệ"

    model_data = torch.load(path, map_location=torch.device("cpu"))

    print(f"Các mô hình được huấn luyện trên các ứng dụng khác nhau có thể đem lại các thông tin khác nhau hoặc không thể đọc!")

    epochs = model_data.get("epoch", None)

    if epochs is None:
        epochs = model_data.get("info", None)
        epoch = epochs.replace("epoch", "").replace("e", "").isdigit()

        if epoch and epochs is None: epochs = "Không tìm thấy kỷ nguyên"

    steps = model_data.get("step", "Không tìm thấy")

    sr = model_data.get("sr", "Không tìm thấy tốc độ lấy mẫu")
    f0 = model_data.get("f0", "Không tìm thấy huấn luyện cao độ")

    version = model_data.get("version", "Không tìm thấy phiên bản")
    creation_date = model_data.get("creation_date", "Không tìm thấy thời gian tạo")
    model_hash = model_data.get("model_hash", "Không tìm thấy")

    pitch_guidance = "Được huấn luyện cao độ" if f0 else "Không được huấn luyện cao độ"

    creation_date_str = prettify_date(creation_date) if creation_date else "Không tìm thấy thời gian tạo"

    model_name = model_data.get("model_name", "Mô hình không được ghi chép")
    model_author = model_data.get("author", "Mô hình không được ghi chép")

    print("Hoàn thành")

    return (
        f"Tên mô hình: {model_name}\n"
        f"Người tạo mô hình: {model_author}\n"
        f"Kỷ nguyên: {epochs}\n"
        f"Số bước: {steps}\n"
        f"Phiên bản của mô hình: {version}\n"
        f"Tốc độ lấy mẫu: {sr}\n"
        f"Huấn luyện cao độ: {pitch_guidance}\n"
        f"Hash (ID): {model_hash}\n"
        f"Thời gian tạo: {creation_date_str}\n"
    )

if not duong_dan_mo_hinh:
    uploaded = files.upload()
    duong_dan_mo_hinh = list(uploaded.keys())[0]

%cd /content/Vietnamese_RVC
print(model_info(duong_dan_mo_hinh))

# **Tách nhạc 🎼**

In [ ]:
#@title **Tách Nhạc**
import os
import re
import shutil
import yt_dlp
import warnings

from ipywidgets import Button
from google.colab import files
from urllib.parse import urlparse
from IPython.display import clear_output, display

def download_url(url):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        ydl_opts = {"format": "bestaudio/best", "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": "wav", "preferredquality": "192"}], "quiet": True, "no_warnings": True, "noplaylist": True, "verbose": False}

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output = os.path.join("audios", re.sub(r'\s+', '-', re.sub(r'[^\w\s\u4e00-\u9fff\uac00-\ud7af\u0400-\u04FF\u1100-\u11FF]', '', ydl.extract_info(url, download=False).get('title', 'video')).strip()))
            if os.path.exists(audio_output): shutil.rmtree(audio_output, ignore_errors=True)

            ydl_opts['outtmpl'] = audio_output.replace(".wav", "")

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            audio_output = audio_output + ".wav"
            if os.path.exists(audio_output): os.remove(audio_output)

            ydl.download([url])

        return audio_output

def separator_music(input, format, segments_size, overlap, separator_model, backing, reverb):
    filename, _ = os.path.splitext(os.path.basename(input))
    output = os.path.join("audios", filename)

    if not os.path.exists(output): os.makedirs(output)
    backing_reverb = backing and reverb

    !python main/inference/separator_music.py --input_path $input --output_path $output --format $format --shifts 2 --segments_size $segments_size --overlap $overlap --mdx_hop_length 1024 --mdx_batch_size 1 --clean_audio False --clean_strength 0.7 --kara_model 'Version-2' --backing $backing --mdx_denoise True --reverb $reverb --backing_reverb $backing_reverb --model_name $separator_model --sample_rate 44100

def is_url(path):
    try:
        result = urlparse(path)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

#@markdown **Các tùy chọn về tách bè và tách vang khi tách nhạc**
tach_be = False # @param {"type":"boolean"}
tach_vang = False # @param {"type":"boolean"}
#@markdown **Đường dẫn đầu vào (có thể là đường dẫn liên kết hoặc đường dẫn tệp) và định dạng đầu ra tệp**
duong_dan = "" #@param {"type":"string", "placeholder":"Nhập đường dẫn đến tệp âm thanh hoặc đường dẫn liên kết âm thanh"}
dinh_dang_am_thanh = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Các tùy chọn về mô hình tách nhạc, mức chồng chéo và kích thước phân đoạn**
mo_hinh_tach_nhac = "Voc_FT" #@param ["Main_340", "Main_390", "Main_406", "Main_427", "Main_438", "Inst_full_292", "Inst_HQ_1", "Inst_HQ_2", "Inst_HQ_3", "Inst_HQ_4", "Inst_HQ_5", "Kim_Vocal_1", "Kim_Vocal_2", "Kim_Inst", "Inst_187_beta", "Inst_82_beta", "Inst_90_beta", "Voc_FT", "Crowd_HQ", "Inst_1", "Inst_2", "Inst_3", "MDXNET_1_9703", "MDXNET_2_9682", "MDXNET_3_9662", "Inst_Main", "MDXNET_Main", "MDXNET_9482", "HT-Normal", "HT-Tuned", "HD_MMI",  "HT_6S"]
chong_cheo = "0.25" # @param ["0.25", "0.5", "0.75", "0.99"]
kich_thuoc_phan_doan = 256 # @param {"type":"slider","min":32,"max":2048,"step":8}

%cd /content/Vietnamese_RVC/

if not duong_dan:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = download_url(duong_dan) if is_url(duong_dan) else duong_dan

separator_music(input_path, dinh_dang_am_thanh, kich_thuoc_phan_doan, chong_cheo, mo_hinh_tach_nhac, tach_be, tach_vang)
clear_output()

filename, _ = os.path.splitext(os.path.basename(input_path))
print(f"Tệp đầu ra của bạn nằm trong thư mục: /content/Vietnamese_RVC/audios/{filename}")

display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

# **Chuyển đổi âm thanh 🔊**

In [ ]:
#@title **🔎 Tìm kiếm mô hình**
import json
import codecs
import requests
from bs4 import BeautifulSoup

def fetch_models_data(search):
    all_table_data = []
    page = 1

    while 1:
        try:
            response = requests.post(url=codecs.decode("uggcf://ibvpr-zbqryf.pbz/srgpu_qngn.cuc", "rot13"), data={"page": page, "search": search})

            if response.status_code == 200:
                table_data = response.json().get("table", "")
                if not table_data.strip(): break
                all_table_data.append(table_data)
                page += 1
            else: raise Exception(f"Đã xảy ra lỗi, Mã lỗi: {response.status_code}")
        except json.JSONDecodeError:
            raise Exception("Đã xảy ra lỗi không thể phân tích từ phản hồi.")
        except requests.RequestException as e:
            raise Exception(f"Gửi yêu cầu thất bại: {e}")
    return all_table_data

def search_models(name):
    tables = fetch_models_data(name)

    if len(tables) == 0: print("Không tìm thấy...")
    else:
        for table in tables:
            for row in BeautifulSoup(table, "html.parser").select("tr"):
                name_tag, url_tag = row.find("a", {"class": "fs-5"}), row.find("a", {"class": "btn btn-sm fw-bold btn-light ms-0 p-1 ps-2 pe-2"})
                if name_tag and url_tag:
                    name = name_tag.text.replace(".pth", "").replace(".index", "").replace(".zip", "").replace(" ", "_").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(",", "").replace('"', "").replace("'", "").replace("|", "").strip()
                    url = url_tag["href"].replace("https://easyaivoice.com/run?url=", "")
                    print(f"{name}: {url}")

#@markdown **Tìm kiếm liên kết mô hình**
ten_mo_hinh = "" # @param {"type":"string","placeholder":"Tên của mô hình cần tìm"}
search_models(ten_mo_hinh)

In [ ]:
#@title **📩 Tải xuống mô hình**
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

def move_files_from_directory(src_dir, dest_weights, dest_logs, model_name):
    for root, _, files in os.walk(src_dir):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                model_log_dir = os.path.join(dest_logs, model_name)
                os.makedirs(model_log_dir, exist_ok=True)

                filepath = os.path.join(model_log_dir, file.replace(' ', '_').replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace(",", "").replace('"', "").replace("'", "").replace("|", "").strip())
                if os.path.exists(filepath): os.remove(filepath)
                shutil.move(file_path, filepath)
            elif file.endswith(".pth") and not file.startswith("D_") and not file.startswith("G_"):
                pth_path = os.path.join(dest_weights, model_name + ".pth")
                if os.path.exists(pth_path): os.remove(pth_path)
                shutil.move(file_path, pth_path)

def download_model(url=None, model=None):
    if not url: raise ValueError("Hãy cung cấp đường dẫn liên kết mô hình hợp lệ!")
    if not model: raise ValueError("Hãy cung cấp tên mô hình để tải về!")

    model = model.replace(".pth", "").replace(".index", "").replace(".zip", "").replace(" ", "_").replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(",", "").replace('"', "").replace("'", "").replace("|", "").strip()
    url = url.replace("/blob/", "/resolve/").replace("?download=true", "").strip()

    download_dir = os.path.join("download_model")
    weights_dir = os.path.join("assets", "weights")
    logs_dir = os.path.join("assets", "logs")

    if not os.path.exists(download_dir): os.makedirs(download_dir, exist_ok=True)
    if not os.path.exists(weights_dir): os.makedirs(weights_dir, exist_ok=True)
    if not os.path.exists(logs_dir): os.makedirs(logs_dir, exist_ok=True)

    try:
        from main.tools import gdown, meganz, mediafire, pixeldrain, huggingface

        if url.endswith(".pth"): huggingface.HF_download_file(url, os.path.join(weights_dir, f"{model}.pth"))
        elif url.endswith(".index"):
            model_log_dir = os.path.join(logs_dir, model)
            os.makedirs(model_log_dir, exist_ok=True)

            huggingface.HF_download_file(url, os.path.join(model_log_dir, f"{model}.index"))
        elif url.endswith(".zip"):
            output_path = huggingface.HF_download_file(url, os.path.join(download_dir, model + ".zip"))
            shutil.unpack_archive(output_path, download_dir)

            move_files_from_directory(download_dir, weights_dir, logs_dir, model)
        else:
            if "drive.google.com" in url or "drive.usercontent.google.com" in url:
                file_id = None

                if "/file/d/" in url: file_id = url.split("/d/")[1].split("/")[0]
                elif "open?id=" in url: file_id = url.split("open?id=")[1].split("/")[0]
                elif "/download?id=" in url: file_id = url.split("/download?id=")[1].split("&")[0]

                if file_id:
                    file = gdown.gdown_download(id=file_id, output=download_dir)
                    if file.endswith(".zip"): shutil.unpack_archive(file, download_dir)

                    move_files_from_directory(download_dir, weights_dir, logs_dir, model)
            elif "mega.nz" in url:
                meganz.mega_download_url(url, download_dir)

                file_download = next((f for f in os.listdir(download_dir)), None)
                if file_download.endswith(".zip"): shutil.unpack_archive(os.path.join(download_dir, file_download), download_dir)

                move_files_from_directory(download_dir, weights_dir, logs_dir, model)
            elif "mediafire.com" in url:
                file = mediafire.Mediafire_Download(url, download_dir)
                if file.endswith(".zip"): shutil.unpack_archive(file, download_dir)

                move_files_from_directory(download_dir, weights_dir, logs_dir, model)
            elif "pixeldrain.com" in url:
                file = pixeldrain.pixeldrain(url, download_dir)
                if file.endswith(".zip"): shutil.unpack_archive(file, download_dir)

                move_files_from_directory(download_dir, weights_dir, logs_dir, model)
            else: raise ValueError("Liên kết không được hỗ trợ")
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        raise Exception(f"Đã xảy ra lỗi: {e}")
    finally:
        shutil.rmtree(download_dir, ignore_errors=True)

def save_drop_model(dropbox):
    weight_folder = os.path.join("assets", "weights")
    logs_folder = os.path.join("assets", "logs")
    save_model_temp = os.path.join("save_model_temp")

    if not os.path.exists(weight_folder): os.makedirs(weight_folder, exist_ok=True)
    if not os.path.exists(logs_folder): os.makedirs(logs_folder, exist_ok=True)
    if not os.path.exists(save_model_temp): os.makedirs(save_model_temp, exist_ok=True)

    shutil.move(dropbox, save_model_temp)

    try:
        file_name = os.path.basename(dropbox)

        if file_name.endswith(".pth") and file_name.endswith(".index"): raise ValueError("Tệp bạn vừa tải lên không phải mô hình!")
        else:
            if file_name.endswith(".zip"):
                shutil.unpack_archive(os.path.join(save_model_temp, file_name), save_model_temp)
                move_files_from_directory(save_model_temp, weight_folder, logs_folder, file_name.replace(".zip", ""))
            elif file_name.endswith(".pth"):
                output_file = os.path.join(weight_folder, file_name)
                if os.path.exists(output_file): os.remove(output_file)

                shutil.move(os.path.join(save_model_temp, file_name), output_file)
            elif file_name.endswith(".index"):
                def extract_name_model(filename):
                    match = re.search(r"([A-Za-z]+)(?=_v|\.|$)", filename)
                    return match.group(1) if match else None

                model_logs = os.path.join(logs_folder, extract_name_model(file_name))
                if not os.path.exists(model_logs): os.makedirs(model_logs, exist_ok=True)
                shutil.move(os.path.join(save_model_temp, file_name), model_logs)
            else: raise Exception("Không phân tích được mô hình!")
    except Exception as e:
        raise Exception(f"Đã xảy ra lỗi {e}")
    finally:
        shutil.rmtree(save_model_temp, ignore_errors=True)

#@markdown **Hỗ trợ các liên kết đến từ huggingface.co / drive.google.com / mega.nz / mediafire.com**

ten_cua_mo_hinh = "" # @param {"type":"string","placeholder":"Tên mô hình"}
lien_ket_mo_hinh = "" # @param {"type":"string","placeholder":"https://huggingface.co//..."}

if not lien_ket_mo_hinh:
    uploaded = files.upload()
    save_drop_model(list(uploaded.keys())[0])
else:
    %cd /content/Vietnamese_RVC

    download_model(lien_ket_mo_hinh, ten_cua_mo_hinh)

clear_output()
display(Button(description="\u2714 Hoàn tất!!", button_style="success"))

In [ ]:
#@title 🎵 **Chuyển Đổi Âm Thanh** 🎵
import os
import shutil

from ipywidgets import Button
from pydub import AudioSegment
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC
from main.library.utils import pydub_convert

def convert(pitch, filter_radius, index_rate, volume_envelope, protect, hop_length, f0_method, input_path, output_path, pth_path, index_path, f0_autotune, clean_audio, clean_strength, export_format, embedder_model, resample_sr, split_audio, f0_autotune_strength, checkpointing):
    !python main/inference/convert.py --pitch $pitch --filter_radius $filter_radius --index_rate $index_rate --volume_envelope $volume_envelope --protect $protect --hop_length $hop_length --f0_method $f0_method --input_path $input_path --output_path $output_path --pth_path $pth_path --index_path $index_path --f0_autotune $f0_autotune --clean_audio $clean_audio --clean_strength $clean_strength --export_format $export_format --embedder_model $embedder_model --resample_sr $resample_sr --split_audio $split_audio --f0_autotune_strength $f0_autotune_strength --checkpointing $checkpointing

def convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, input_audio_name, checkpointing, apply_effect):
    if use_original:
        if convert_backing: raise ValueError("Tắt chuyển đổi giọng bè để có thể sử dụng chuyển đổi giọng gốc!")
        elif not_merge_backing: raise ValueError("Tắt kết hợp bè để có thể sử dụng chuyển đổi giọng gốc!")

    if not model or not os.path.exists(model) or os.path.isdir(model) or not model.endswith(".pth"): raise FileExistsError("Vui lòng cung cấp tệp mô hình hợp lệ!")

    f0method = method if method != "hybrid" else hybrid_method
    embedder_model = embedders if embedders != "custom" else custom_embedders

    if use_audio:
        output_audio = os.path.join("audios", input_audio_name)

        def get_audio_file(label):
            matching_files = [f for f in os.listdir(output_audio) if label in f]

            if not matching_files: return None
            return os.path.join(output_audio, matching_files[0])

        output_path = os.path.join(output_audio, f"Convert_Vocals.{format}")
        output_path_effect = os.path.join(output_audio, f"Convert_Vocal_Effects.{format}")
        output_backing = os.path.join(output_audio, f"Convert_Backing.{format}")
        output_merge_backup = os.path.join(output_audio, f"Vocals+Backing.{format}")
        output_merge_instrument = os.path.join(output_audio, f"Vocals+Instruments.{format}")

        if os.path.exists(output_audio): os.makedirs(output_audio, exist_ok=True)
        if os.path.exists(output_path): os.remove(output_path)

        if use_original:
            original_vocal = get_audio_file('Original_Vocals_No_Reverb.')

            if original_vocal is None: original_vocal = get_audio_file('Original_Vocals.')
            if original_vocal is None: raise FileNotFoundError("Không tìm thấy giọng gốc!")

            input_path = original_vocal
        else:
            main_vocal = get_audio_file('Main_Vocals_No_Reverb.')
            backing_vocal = get_audio_file('Backing_Vocals_No_Reverb.')

            if main_vocal is None: main_vocal = get_audio_file('Main_Vocals.')
            if not not_merge_backing and backing_vocal is None: backing_vocal = get_audio_file('Backing_Vocals.')

            if main_vocal is None: raise FileNotFoundError("Không tìm thấy giọng chính!")
            if not not_merge_backing and backing_vocal is None: raise FileNotFoundError("Không tìm thấy giọng bè")

            input_path = main_vocal
            backing_path = backing_vocal

        print("Bắt đầu chuyển đổi giọng...")

        convert(pitch, filter_radius, index_rate, volume_envelope, protect, hop_length, f0method, input_path, output_path, model, index, autotune, clean, clean_strength, format, embedder_model, resample_sr, split_audio, f0_autotune_strength, checkpointing)

        if apply_effect:
            audio_effects(output_path, output_path_effect, format)
            output_path = output_path_effect

        print("Chuyển đổi giọng hoàn tất!")

        if convert_backing:
            if os.path.exists(output_backing): os.remove(output_backing)

            print("Bắt đầu chuyển đổi giọng bè...")

            convert(pitch, filter_radius, index_rate, volume_envelope, protect, hop_length, f0method, backing_path, output_backing, model, index, autotune, clean, clean_strength, format, embedder_model, resample_sr, split_audio, f0_autotune_strength, checkpointing)

            print("Chuyển đổi giọng bè hoàn tất!")

        if not not_merge_backing and not use_original:
            backing_source = output_backing if convert_backing else backing_vocal

            if os.path.exists(output_merge_backup): os.remove(output_merge_backup)

            pydub_convert(AudioSegment.from_file(output_path)).overlay(pydub_convert(AudioSegment.from_file(backing_source))).export(output_merge_backup, format=format)

        if merge_instrument:
            vocals = output_merge_backup if not not_merge_backing and not use_original else output_path

            if os.path.exists(output_merge_instrument): os.remove(output_merge_instrument)

            instruments = get_audio_file('Instruments.')

            if instruments is None: output_merge_instrument = None
            else: pydub_convert(AudioSegment.from_file(instruments)).overlay(pydub_convert(AudioSegment.from_file(vocals))).export(output_merge_instrument, format=format)
    else:
        if not input or not os.path.exists(input): raise FileNotFoundError("Đường dẫn tệp đầu vào không hợp lệ!")
        if not output: raise ValueError("Đường dẫn tệp đầu ra không hợp lệ!")

        if os.path.isdir(input):
            print("Đường dẫn đầu vào là thư mục, chuyển đổi hàng loạt được kích hoạt...")
            if not [f for f in os.listdir(input) if f.lower().endswith(("wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"))]: raise ValueError("Thư mục đầu vào trống!")

            print("Bắt đầu chuyển đổi hàng loạt...")
            output_dir = os.path.dirname(output) or output

            convert(pitch, filter_radius, index_rate, volume_envelope, protect, hop_length, f0method, input, output_dir, model, index, autotune, clean, clean_strength, format, embedder_model, resample_sr, split_audio, f0_autotune_strength, checkpointing)
            print("Chuyển đổi hàng loạt đã hoàn tất!")
        else:
            output_dir = os.path.dirname(output) or output

            if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)
            if os.path.exists(output): os.remove(output)

            print("Bắt đầu chuyển đổi tệp đầu vào...")
            convert(pitch, filter_radius, index_rate, volume_envelope, protect, hop_length, f0method, input, output, model, index, autotune, clean, clean_strength, format, embedder_model, resample_sr, split_audio, f0_autotune_strength, checkpointing)

            if apply_effect:
                filename, _ = os.path.splitext(os.path.basename(output))
                audio_effects(output, os.path.join(output_dir, f"{filename}_effect.{format}"), format)

            print("Hoàn tất chuyển đổi!")

def convert_selection(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, inputp, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, checkpointing, apply_effect):
    if use_audio:
        choice = [f for f in os.listdir("audios") if os.path.isdir(os.path.join("audios", f))]

        if len(choice) == 0: raise FileNotFoundError("Không tìm thấy bản tách nhạc nào!")
        elif len(choice) == 1:
            choice_audio = choice[0]
            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, apply_effect)
        else:
            print("Đã tìm thấy nhiều hơn 1 bản tách nhạc, vui lòng chọn bài để tiến hành chuyển đổi!")
            for c in choice:
                print(f"{choice.index(c)}. {c}")

            while 1:
                try:
                    choice_select = int(input("Hãy nhập số thứ tự của bản tách: "))

                    if 0 <= choice_select < len(choice):
                        choice_audio = choice[choice_select]
                        print(f"Bạn đã chọn bản tách: {choice_audio}")
                        break
                    else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
                except ValueError:
                    print("Vui lòng nhập một số nguyên.")
                except IndexError:
                    print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

            convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, None, None, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, choice_audio, checkpointing, apply_effect)
    else: convert_audio(clean, autotune, use_audio, use_original, convert_backing, not_merge_backing, merge_instrument, pitch, clean_strength, model, index, index_rate, inputp, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, None, checkpointing, apply_effect)
    return (f"/content/Vietnamese_RVC/audios/{choice_audio}/Vocals+Instruments.{format}" if merge_instrument else (f"/content/Vietnamese_RVC/audios/{choice_audio}/Vocals+Backing.{format}" if convert_backing else (f"/content/Vietnamese_RVC/audios/{choice_audio}/Convert_Vocal_Effects.{format}" if apply_effect else f"/content/Vietnamese_RVC/audios/{choice_audio}/Convert_Vocals.{format}"))) if use_audio else (f"/content/Vietnamese_RVC/audios/output_effect.{format}" if apply_effect else f"/content/Vietnamese_RVC/audios/output.{format}")

def audio_effects(input_path, output_path, export_format):
    if not input_path or not os.path.exists(input_path) or os.path.isdir(input_path): raise FileNotFoundError("Đường dẫn tệp đầu vào không hợp lệ!")
    if not output_path: raise ValueError("Đường dẫn tệp đầu ra không hợp lệ!")
    output_dir = os.path.dirname(output_path) or output_path

    if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)
    if os.path.exists(output_path): os.remove(output_path)

    !python main/inference/audio_effects.py --input_path $input_path --output_path $output_path --reverb_room_size 0.15 --reverb_damping 0.7 --reverb_wet_level 0.2 --reverb_dry_level 0.8 --reverb_width 1.0 --reverb_freeze_mode False --export_format $export_format --reverb True

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

#@markdown **Các tùy chọn sử dụng các bản được tách nhạc được tách trước đó**
su_dung_giong_goc = False #@param {"type":"boolean"}
chuyen_doi_giong_be = False #@param {"type":"boolean"}
ket_hop_nhac_nen = False #@param {"type":"boolean"}
#@markdown **Các tùy chọn về mô hình như: Cao độ và Chỉ mục**
ten_mo_hinh = "" #@param {"type":"string","placeholder":"Tên mô hình"}
cao_do = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
anh_huong_cua_chi_muc = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Tùy chọn đầu vào và định dạng đầu ra của tệp**
duong_dan_dau_vao = "" #@param {"type":"string","placeholder":"Đường dẫn tệp âm thanh"}
dinh_dang_tep = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]
#@markdown **Các tùy chọn về trích xuất dữ liệu, hop length, bảo vệ âm thanh và tự động điều chỉnh trích xuất để đạt chất lượng cao hơn**
phuong_phap_trich_xuat = "rmvpe" #@param ["pm", "dio", "mangio-crepe-tiny", "mangio-crepe-tiny-onnx", "mangio-crepe-small", "mangio-crepe-small-onnx", "mangio-crepe-medium", "mangio-crepe-medium-onnx", "mangio-crepe-large", "mangio-crepe-large-onnx", "mangio-crepe-full", "mangio-crepe-full-onnx", "crepe-tiny", "crepe-tiny-onnx", "crepe-small", "crepe-small-onnx", "crepe-medium", "crepe-medium-onnx", "crepe-large", "crepe-large-onnx", "crepe-full", "crepe-full-onnx", "fcpe", "fcpe-onnx", "fcpe-legacy", "fcpe-legacy-onnx", "rmvpe", "rmvpe-onnx", "rmvpe-legacy", "rmvpe-legacy-onnx", "harvest", "yin", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
bao_ve_phu_am = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
tu_dong_dieu_chinh = False #@param {"type":"boolean"}
#@markdown **Tùy chọn về cắt âm thanh tĩnh để đạt tốc độ cao hơn và áp dụng hiệu ứng vang vào âm thanh**
cat_am_thanh = False #@param {"type":"boolean"}
ap_dung_hieu_ung = False #@param {"type":"boolean"}

if ten_mo_hinh:
    model_path = os.path.join("assets", "weights", f"{ten_mo_hinh}.pth")
    index_path = get_index(f"{ten_mo_hinh}.pth".split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith(".pth") and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Nhập số thứ tự của mô hình: "))
                if model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"Bạn đã chọn mô hình: {selected_model}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")

if not duong_dan_dau_vao and not (su_dung_giong_goc or chuyen_doi_giong_be or ket_hop_nhac_nen):
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = os.path.join("audios", os.path.basename(filename).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
    shutil.move(filename, input_path)
else: input_path = duong_dan_dau_vao

output_files = convert_selection(False, tu_dong_dieu_chinh, (su_dung_giong_goc or chuyen_doi_giong_be or ket_hop_nhac_nen), su_dung_giong_goc, chuyen_doi_giong_be, False, ket_hop_nhac_nen, cao_do, 0.7, model_path, index_path, anh_huong_cua_chi_muc, input_path, os.path.join("audios", "output.wav"), dinh_dang_tep, phuong_phap_trich_xuat, None, hop_length, "contentvec_base", None, 0, 3, 1, bao_ve_phu_am, cat_am_thanh, 1, False, ap_dung_hieu_ung)
clear_output()

print(f"Tệp đầu ra của bạn là: {output_files}")
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🎵 **Chuyển Đổi Văn Bản** 🎵
import os

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

%cd /content/Vietnamese_RVC

from main.tools import edge_tts

async def TTS(prompt, voice, speed, output, pitch):
    if not prompt: raise ValueError("Vui lòng cung cấp văn bản để đọc!")
    if not voice: raise ValueError("Vui lòng chọn giọng nói để chuyển đổi!")
    if not output: raise ValueError("Vui lòng cung cấp đầu vào hợp lệ!")

    if os.path.isdir(output): output = os.path.join(output, f"tts.wav")

    output_dir = os.path.dirname(output) or output
    if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)

    await edge_tts.Communicate(text=prompt, voice=voice, rate=f"+{speed}%" if speed >= 0 else f"{speed}%", pitch=f"+{pitch}Hz" if pitch >= 0 else f"{pitch}Hz").save(output)
    return output

def convert_tts(clean, autotune, pitch, clean_strength, model, index, index_rate, input, output, format, method, hybrid_method, hop_length, embedders, custom_embedders, resample_sr, filter_radius, volume_envelope, protect, split_audio, f0_autotune_strength, checkpointing):
    if not model or not os.path.exists(model) or os.path.isdir(model) or not model.endswith(".pth"): raise FileExistsError("Vui lòng cung cấp tệp mô hình hợp lệ")
    if not input or not os.path.exists(input): raise FileNotFoundError("Đường dẫn tệp đầu vào không hợp lệ!")
    if not output: raise ValueError("Đường dẫn tệp đầu ra không hợp lệ!")

    if os.path.isdir(output): output = os.path.join(output, f"tts-convert.{format}")

    output_dir = os.path.dirname(output)
    if not os.path.exists(output_dir): os.makedirs(output_dir, exist_ok=True)

    if os.path.exists(output): os.remove(output)

    f0method = method if method != "hybrid" else hybrid_method
    embedder_model = embedders if embedders != "custom" else custom_embedders

    !python main/inference/convert.py --pitch $pitch --filter_radius $filter_radius --index_rate $index_rate --volume_envelope $volume_envelope --protect $protect --hop_length $hop_length --f0_method $f0method --input_path $input --output_path $output --pth_path $model --index_path $index --f0_autotune $autotune --clean_audio $clean --clean_strength $clean_strength --export_format $format --embedder_model $embedder_model --resample_sr $resample_sr --split_audio $split_audio --f0_autotune_strength $f0_autotune_strength --checkpointing $checkpointing

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

def process_input(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        file_contents = file.read()

    return file_contents

#@markdown **Văn bản cần đọc và các tùy chọn về giọng đọc và tốc độ đọc**
van_ban = "" # @param {"type":"string", "placeholder":"Văn bản cần đọc"}
giong_noi = "vi-VN-NamMinhNeural" # @param ["af-ZA-AdriNeural", "af-ZA-WillemNeural", "sq-AL-AnilaNeural", "sq-AL-IlirNeural", "am-ET-AmehaNeural", "am-ET-MekdesNeural", "ar-DZ-AminaNeural", "ar-DZ-IsmaelNeural", "ar-BH-AliNeural", "ar-BH-LailaNeural", "ar-EG-SalmaNeural", "ar-EG-ShakirNeural", "ar-IQ-BasselNeural", "ar-IQ-RanaNeural", "ar-JO-SanaNeural", "ar-JO-TaimNeural", "ar-KW-FahedNeural", "ar-KW-NouraNeural", "ar-LB-LaylaNeural", "ar-LB-RamiNeural", "ar-LY-ImanNeural", "ar-LY-OmarNeural", "ar-MA-JamalNeural", "ar-MA-MounaNeural", "ar-OM-AbdullahNeural", "ar-OM-AyshaNeural", "ar-QA-AmalNeural", "ar-QA-MoazNeural", "ar-SA-HamedNeural", "ar-SA-ZariyahNeural", "ar-SY-AmanyNeural", "ar-SY-LaithNeural", "ar-TN-HediNeural", "ar-TN-ReemNeural", "ar-AE-FatimaNeural", "ar-AE-HamdanNeural", "ar-YE-MaryamNeural", "ar-YE-SalehNeural", "az-AZ-BabekNeural", "az-AZ-BanuNeural", "bn-BD-NabanitaNeural", "bn-BD-PradeepNeural", "bn-IN-BashkarNeural", "bn-IN-TanishaaNeural", "bs-BA-GoranNeural", "bs-BA-VesnaNeural", "bg-BG-BorislavNeural", "bg-BG-KalinaNeural", "my-MM-NilarNeural", "my-MM-ThihaNeural", "ca-ES-EnricNeural", "ca-ES-JoanaNeural", "zh-HK-HiuGaaiNeural", "zh-HK-HiuMaanNeural", "zh-HK-WanLungNeural", "zh-CN-XiaoxiaoNeural", "zh-CN-XiaoyiNeural", "zh-CN-YunjianNeural", "zh-CN-YunxiNeural", "zh-CN-YunxiaNeural", "zh-CN-YunyangNeural", "zh-CN-liaoning-XiaobeiNeural", "zh-TW-HsiaoChenNeural", "zh-TW-YunJheNeural", "zh-TW-HsiaoYuNeural", "zh-CN-shaanxi-XiaoniNeural", "hr-HR-GabrijelaNeural", "hr-HR-SreckoNeural", "cs-CZ-AntoninNeural", "cs-CZ-VlastaNeural", "da-DK-ChristelNeural", "da-DK-JeppeNeural", "nl-BE-ArnaudNeural", "nl-BE-DenaNeural", "nl-NL-ColetteNeural", "nl-NL-FennaNeural", "nl-NL-MaartenNeural", "en-AU-NatashaNeural", "en-AU-WilliamNeural", "en-CA-ClaraNeural", "en-CA-LiamNeural", "en-HK-SamNeural", "en-HK-YanNeural", "en-IN-NeerjaExpressiveNeural", "en-IN-NeerjaNeural", "en-IN-PrabhatNeural", "en-IE-ConnorNeural", "en-IE-EmilyNeural", "en-KE-AsiliaNeural", "en-KE-ChilembaNeural", "en-NZ-MitchellNeural", "en-NZ-MollyNeural", "en-NG-AbeoNeural", "en-NG-EzinneNeural", "en-PH-JamesNeural", "en-PH-RosaNeural", "en-SG-LunaNeural", "en-SG-WayneNeural", "en-ZA-LeahNeural", "en-ZA-LukeNeural", "en-TZ-ElimuNeural", "en-TZ-ImaniNeural", "en-GB-LibbyNeural", "en-GB-MaisieNeural", "en-GB-RyanNeural", "en-GB-SoniaNeural", "en-GB-ThomasNeural", "en-US-AvaMultilingualNeural", "en-US-AndrewMultilingualNeural", "en-US-EmmaMultilingualNeural", "en-US-BrianMultilingualNeural", "en-US-AvaNeural", "en-US-AndrewNeural", "en-US-EmmaNeural", "en-US-BrianNeural", "en-US-AnaNeural", "en-US-AriaNeural", "en-US-ChristopherNeural", "en-US-EricNeural", "en-US-GuyNeural", "en-US-JennyNeural", "en-US-MichelleNeural", "en-US-RogerNeural", "en-US-SteffanNeural", "et-EE-AnuNeural", "et-EE-KertNeural", "fil-PH-AngeloNeural", "fil-PH-BlessicaNeural", "fi-FI-HarriNeural", "fi-FI-NooraNeural", "fr-BE-CharlineNeural", "fr-BE-GerardNeural", "fr-CA-ThierryNeural", "fr-CA-AntoineNeural", "fr-CA-JeanNeural", "fr-CA-SylvieNeural", "fr-FR-VivienneMultilingualNeural", "fr-FR-RemyMultilingualNeural", "fr-FR-DeniseNeural", "fr-FR-EloiseNeural", "fr-FR-HenriNeural", "fr-CH-ArianeNeural", "fr-CH-FabriceNeural", "gl-ES-RoiNeural", "gl-ES-SabelaNeural", "ka-GE-EkaNeural", "ka-GE-GiorgiNeural", "de-AT-IngridNeural", "de-AT-JonasNeural", "de-DE-SeraphinaMultilingualNeural", "de-DE-FlorianMultilingualNeural", "de-DE-AmalaNeural", "de-DE-ConradNeural", "de-DE-KatjaNeural", "de-DE-KillianNeural", "de-CH-JanNeural", "de-CH-LeniNeural", "el-GR-AthinaNeural", "el-GR-NestorasNeural", "gu-IN-DhwaniNeural", "gu-IN-NiranjanNeural", "he-IL-AvriNeural", "he-IL-HilaNeural", "hi-IN-MadhurNeural", "hi-IN-SwaraNeural", "hu-HU-NoemiNeural", "hu-HU-TamasNeural", "is-IS-GudrunNeural", "is-IS-GunnarNeural", "id-ID-ArdiNeural", "id-ID-GadisNeural", "ga-IE-ColmNeural", "ga-IE-OrlaNeural", "it-IT-GiuseppeNeural", "it-IT-DiegoNeural", "it-IT-ElsaNeural", "it-IT-IsabellaNeural", "ja-JP-KeitaNeural", "ja-JP-NanamiNeural", "jv-ID-DimasNeural", "jv-ID-SitiNeural", "kn-IN-GaganNeural", "kn-IN-SapnaNeural", "kk-KZ-AigulNeural", "kk-KZ-DauletNeural", "km-KH-PisethNeural", "km-KH-SreymomNeural", "ko-KR-HyunsuNeural", "ko-KR-InJoonNeural", "ko-KR-SunHiNeural", "lo-LA-ChanthavongNeural", "lo-LA-KeomanyNeural", "lv-LV-EveritaNeural", "lv-LV-NilsNeural", "lt-LT-LeonasNeural", "lt-LT-OnaNeural", "mk-MK-AleksandarNeural", "mk-MK-MarijaNeural", "ms-MY-OsmanNeural", "ms-MY-YasminNeural", "ml-IN-MidhunNeural", "ml-IN-SobhanaNeural", "mt-MT-GraceNeural", "mt-MT-JosephNeural", "mr-IN-AarohiNeural", "mr-IN-ManoharNeural", "mn-MN-BataaNeural", "mn-MN-YesuiNeural", "ne-NP-HemkalaNeural", "ne-NP-SagarNeural", "nb-NO-FinnNeural", "nb-NO-PernilleNeural", "ps-AF-GulNawazNeural", "ps-AF-LatifaNeural", "fa-IR-DilaraNeural", "fa-IR-FaridNeural", "pl-PL-MarekNeural", "pl-PL-ZofiaNeural", "pt-BR-ThalitaNeural", "pt-BR-AntonioNeural", "pt-BR-FranciscaNeural", "pt-PT-DuarteNeural", "pt-PT-RaquelNeural", "ro-RO-AlinaNeural", "ro-RO-EmilNeural", "ru-RU-DmitryNeural", "ru-RU-SvetlanaNeural", "sr-RS-NicholasNeural", "sr-RS-SophieNeural", "si-LK-SameeraNeural", "si-LK-ThiliniNeural", "sk-SK-LukasNeural", "sk-SK-ViktoriaNeural", "sl-SI-PetraNeural", "sl-SI-RokNeural", "so-SO-MuuseNeural", "so-SO-UbaxNeural", "es-AR-ElenaNeural", "es-AR-TomasNeural", "es-BO-MarceloNeural", "es-BO-SofiaNeural", "es-CL-CatalinaNeural", "es-CL-LorenzoNeural", "es-ES-XimenaNeural", "es-CO-GonzaloNeural", "es-CO-SalomeNeural", "es-CR-JuanNeural", "es-CR-MariaNeural", "es-CU-BelkysNeural", "es-CU-ManuelNeural", "es-DO-EmilioNeural", "es-DO-RamonaNeural", "es-EC-AndreaNeural", "es-EC-LuisNeural", "es-SV-LorenaNeural", "es-SV-RodrigoNeural", "es-GQ-JavierNeural", "es-GQ-TeresaNeural", "es-GT-AndresNeural", "es-GT-MartaNeural", "es-HN-CarlosNeural", "es-HN-KarlaNeural", "es-MX-DaliaNeural", "es-MX-JorgeNeural", "es-NI-FedericoNeural", "es-NI-YolandaNeural", "es-PA-MargaritaNeural", "es-PA-RobertoNeural", "es-PY-MarioNeural", "es-PY-TaniaNeural", "es-PE-AlexNeural", "es-PE-CamilaNeural", "es-PR-KarinaNeural", "es-PR-VictorNeural", "es-ES-AlvaroNeural", "es-ES-ElviraNeural", "es-US-AlonsoNeural", "es-US-PalomaNeural", "es-UY-MateoNeural", "es-UY-ValentinaNeural", "es-VE-PaolaNeural", "es-VE-SebastianNeural", "su-ID-JajangNeural", "su-ID-TutiNeural", "sw-KE-RafikiNeural", "sw-KE-ZuriNeural", "sw-TZ-DaudiNeural", "sw-TZ-RehemaNeural", "sv-SE-MattiasNeural", "sv-SE-SofieNeural", "ta-IN-PallaviNeural", "ta-IN-ValluvarNeural", "ta-MY-KaniNeural", "ta-MY-SuryaNeural", "ta-SG-AnbuNeural", "ta-SG-VenbaNeural", "ta-LK-KumarNeural", "ta-LK-SaranyaNeural", "te-IN-MohanNeural", "te-IN-ShrutiNeural", "th-TH-NiwatNeural", "th-TH-PremwadeeNeural", "tr-TR-AhmetNeural", "tr-TR-EmelNeural", "uk-UA-OstapNeural", "uk-UA-PolinaNeural", "ur-IN-GulNeural", "ur-IN-SalmanNeural", "ur-PK-AsadNeural", "ur-PK-UzmaNeural", "uz-UZ-MadinaNeural", "uz-UZ-SardorNeural", "vi-VN-HoaiMyNeural", "vi-VN-NamMinhNeural", "cy-GB-AledNeural", "cy-GB-NiaNeural", "zu-ZA-ThandoNeural", "zu-ZA-ThembaNeural"]
toc_do_doc = 0 # @param {"type":"slider","min":-100,"max":100,"step":1}
#@markdown **Các tùy chọn về mô hình như: Cao độ và Chỉ mục**
ten_mo_hinh = "" #@param {"type":"string","placeholder":"Tên mô hình"}
cao_do = 0 #@param {"type":"slider","min":-20,"max":20,"step":1}
anh_huong_cua_chi_muc = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.01}
#@markdown **Các tùy chọn về trích xuất dữ liệu, hop length, bảo vệ âm thanh và tự động điều chỉnh trích xuất để đạt chất lượng cao hơn**
phuong_phap_trich_xuat = "rmvpe" #@param ["pm", "dio", "mangio-crepe-tiny", "mangio-crepe-tiny-onnx", "mangio-crepe-small", "mangio-crepe-small-onnx", "mangio-crepe-medium", "mangio-crepe-medium-onnx", "mangio-crepe-large", "mangio-crepe-large-onnx", "mangio-crepe-full", "mangio-crepe-full-onnx", "crepe-tiny", "crepe-tiny-onnx", "crepe-small", "crepe-small-onnx", "crepe-medium", "crepe-medium-onnx", "crepe-large", "crepe-large-onnx", "crepe-full", "crepe-full-onnx", "fcpe", "fcpe-onnx", "fcpe-legacy", "fcpe-legacy-onnx", "rmvpe", "rmvpe-onnx", "rmvpe-legacy", "rmvpe-legacy-onnx", "harvest", "yin", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}
bao_ve_phu_am = 0.5 #@param {"type":"slider","min":0,"max":1,"step":0.1}
tu_dong_dieu_chinh = False #@param {"type":"boolean"}
#@markdown **Tùy chọn về cắt âm thanh tĩnh để đạt tốc độ cao hơn và định dạng tệp đầu ra**
cat_am_thanh = False #@param {"type":"boolean"}
dinh_dang_tep = "wav" #@param ["wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3"]

if van_ban: input_text = van_ban
else:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_text = process_input(filename)

if ten_mo_hinh:
    model_path = os.path.join("assets", "weights", f"{ten_mo_hinh}.pth")
    index_path = get_index(f"{ten_mo_hinh}.pth".split("_")[0])
else:
    model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith(".pth") and not model.startswith("G_") and not model.startswith("D_")))
    indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

    if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
    elif len(model_name) == 1:
        model_path = os.path.join("assets", "weights", model_name[0])
        index_path = get_index(os.path.basename(model_name[0])[0])
    else:
        print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index = int(input("Nhập số thứ tự của mô hình: "))
                if 0 <= model_index < len(model_name):
                    selected_model = model_name[model_index]
                    print(f"Bạn đã chọn mô hình: {selected_model}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        model_path = os.path.join("assets", "weights", selected_model)
        index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")

convert_tts(False, tu_dong_dieu_chinh, cao_do, 0.7, model_path, index_path, anh_huong_cua_chi_muc, await TTS(input_text, giong_noi, toc_do_doc, os.path.join("audios", f"tts.{dinh_dang_tep}"), 0), os.path.join("audios", f"tts-convert.{dinh_dang_tep}"), dinh_dang_tep, phuong_phap_trich_xuat, None, hop_length, "contentvec_base", None, 0, 3, 1, bao_ve_phu_am, cat_am_thanh, 1, False)
clear_output()

print(f"Tệp đầu ra của bạn là: /content/Vietnamese_RVC/audios/tts-convert.{dinh_dang_tep}")
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

# **Huấn luyện mô hình 🤖**

In [ ]:
#@title 📩 **Tải xuống mô hình huấn luyện trước**
#@markdown **Chạy ô để có thể lựa chọn mô hình và tốc độ lấy mẫu của mô hình để tiến hành tải xống**
import os
import shutil
import codecs
import requests

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

from main.tools import huggingface

def fetch_pretrained_data():
    response = requests.get(codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/wfba/phfgbz_cergenvarq.wfba", "rot13"))
    response.raise_for_status()
    return response.json()

model_list = list(fetch_pretrained_data().keys())

for m in model_list:
    print(f"{model_list.index(m)}. {m}")

while 1:
    try:
        model_index = int(input("Nhập số thứ tự của mô hình: "))
        if 0 <= model_index < len(model_list):
            selected_model = model_list[model_index]
            clear_output()
            print(f"Bạn đã chọn mô hình: {selected_model}")
            break
        else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
    except ValueError:
        print("Vui lòng nhập một số nguyên.")
    except IndexError:
        print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

model_sr_list = list(fetch_pretrained_data()[selected_model].keys())

if len(model_sr_list) == 1: selected_sr = model_sr_list[0]
else:
    for sr in model_sr_list:
        print(f"{model_sr_list.index(sr)}. {sr}")

    while 1:
        try:
            model_sr_index = int(input("Nhập số thứ tự của tốc độ lấy mẫu: "))
            if 0 <= model_sr_index < len(model_sr_list):
                selected_sr = model_sr_list[model_sr_index]
                print(f"Bạn đã chọn tốc độ lấy mẫu: {selected_sr}")
                clear_output()
                break
            else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
        except ValueError:
            print("Vui lòng nhập một số nguyên.")
        except IndexError:
            print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

paths = fetch_pretrained_data()[selected_model][selected_sr]

pretraineds_custom_path = os.path.join("assets", "models", "pretrained_custom")

if not os.path.exists(pretraineds_custom_path): os.makedirs(pretraineds_custom_path, exist_ok=True)
url = codecs.decode("uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/cergenvarq_phfgbz/", "rot13") + paths

file = huggingface.HF_download_file(url.replace("/blob/", "/resolve/").replace("?download=true", "").strip(), os.path.join(pretraineds_custom_path, paths))

if file.endswith(".zip"):
    shutil.unpack_archive(file, pretraineds_custom_path)
    os.remove(file)

clear_output()
Button(description="\u2714 Hoàn tất!", button_style="success")

In [ ]:
#@title 📁 **Tạo dữ liệu huấn luyện từ đường dẫn**
#@markdown **Tạo dữ liệu huấn luyện từ đường dẫn liên kết**

def create_dataset(input_audio, output_dataset, clean_dataset, clean_strength, separator_reverb, version, overlap, segments_size, denoise_mdx, skip, skip_start, skip_end, hop_length, batch_size, sample_rate):
    !python main/inference/create_dataset.py --input_audio $input_audio --output_dataset $output_dataset --clean_dataset $clean_dataset --clean_strength $clean_strength --separator_reverb $separator_reverb --kim_vocal_version $version --overlap $overlap --segments_size $segments_size --mdx_hop_length $hop_length --mdx_batch_size $batch_size --denoise_mdx $denoise_mdx --skip $skip --skip_start_audios $skip_start --skip_end_audios $skip_end --sample_rate $sample_rate

lam_sach_du_lieu = False #@param {"type":"boolean"}
duong_dan_lien_ket = "" #@param {"type":"string", placeholder:"Đường dẫn liên kết"}

%cd /content/Vietnamese_RVC
create_dataset(duong_dan_lien_ket, "dataset", lam_sach_du_lieu, 0.7, True, 2, 0.25, 256, True, False, 0, 0, 1024, 1, 48000)

In [ ]:
#@title 🔨 **Xử lý trích xuất dữ liệu** ⛏️
import os
import shutil

from ipywidgets import Button
from google.colab import files
from IPython.display import clear_output, display

def preprocess(model_name, sample_rate, cpu_core, cut_preprocess, process_effects, path, clean_dataset, clean_strength):
    dataset = os.path.join(path)
    sr = int(float(sample_rate.rstrip("k")) * 1000)

    if not any(f.lower().endswith(("wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3")) for f in os.listdir(dataset) if os.path.isfile(os.path.join(dataset, f))): raise FileNotFoundError("Không tìm thấy dữ liệu huấn luyện!")

    model_dir = os.path.join("assets", "logs", model_name)
    if os.path.exists(model_dir): shutil.rmtree(model_dir, ignore_errors=True)

    !python main/inference/preprocess.py --model_name $model_name --dataset_path $dataset --sample_rate $sr --cpu_cores $cpu_core --cut_preprocess $cut_preprocess --process_effects $process_effects --clean_dataset $clean_dataset --clean_strength $clean_strength

def extract(model_name, version, method, pitch_guidance, hop_length, cpu_cores, gpu, sample_rate, embedders, custom_embedders):
    embedder_model = embedders if embedders != "custom" else custom_embedders
    sr = int(float(sample_rate.rstrip("k")) * 1000)

    model_dir = os.path.join("assets", "logs", model_name)
    if not any(os.path.isfile(os.path.join(model_dir, "sliced_audios", f)) for f in os.listdir(os.path.join(model_dir, "sliced_audios"))) or not any(os.path.isfile(os.path.join(model_dir, "sliced_audios_16k", f)) for f in os.listdir(os.path.join(model_dir, "sliced_audios_16k"))): raise FileNotFoundError("Không tìm thấy dữ liệu đã được xử lý!")

    !python main/inference/extract.py --model_name $model_name --rvc_version $version --f0_method $method --pitch_guidance $pitch_guidance --hop_length $hop_length --cpu_cores $cpu_cores --gpu $gpu --sample_rate $sr --embedder_model $embedder_model

def create_index(model_name, rvc_version, index_algorithm):
    model_dir = os.path.join("assets", "logs", model_name)
    if not any(os.path.isfile(os.path.join(model_dir, f"{rvc_version}_extracted", f)) for f in os.listdir(os.path.join(model_dir, f"{rvc_version}_extracted"))): raise FileNotFoundError("Không tìm thấy dữ liệu đã được trích xuất")

    !python main/inference/create_index.py --model_name $model_name --rvc_version $rvc_version --index_algorithm $index_algorithm

#@markdown **Tải dữ liệu từ google drive lên dữ liệu của google colab để tiến hành xử lý và trích xuất**
tai_du_lieu_tu_drive = False # @param {"type":"boolean"}
#@markdown **Tên của mô hình để tiến hành**
ten_mo_hinh = "" #@param {"type":"string", placeholder: "Tên mô hình"}
#@markdown **Tốc độ lấy mẫu của mô hình để tiến hành**
toc_do_lay_mau = "48k" #@param ["32k", "40k", "44.1k", "48k"]
#@markdown **Phương pháp trích xuất cao độ và hop length cho mô hình**
phuong_phap_trich_xuat = "rmvpe" #@param ["pm", "dio", "mangio-crepe-tiny", "mangio-crepe-tiny-onnx", "mangio-crepe-small", "mangio-crepe-small-onnx", "mangio-crepe-medium", "mangio-crepe-medium-onnx", "mangio-crepe-large", "mangio-crepe-large-onnx", "mangio-crepe-full", "mangio-crepe-full-onnx", "crepe-tiny", "crepe-tiny-onnx", "crepe-small", "crepe-small-onnx", "crepe-medium", "crepe-medium-onnx", "crepe-large", "crepe-large-onnx", "crepe-full", "crepe-full-onnx", "fcpe", "fcpe-onnx", "fcpe-legacy", "fcpe-legacy-onnx", "rmvpe", "rmvpe-onnx", "rmvpe-legacy", "rmvpe-legacy-onnx", "harvest", "yin", "pyin"]
hop_length = 64 # @param {"type":"slider","min":64,"max":512,"step":1}

%cd /content/Vietnamese_RVC

if tai_du_lieu_tu_drive:
    if not os.path.exists("/content/drive/MyDrive"): raise ValueError("Bạn chưa kết nối với google drive")
    if len([f for f in os.listdir("/content/drive/MyDrive/dataset") if not ".ipynb_checkpoints" in f]) < 1: raise FileNotFoundError("Không tìm thấy dữ liệu")

    for audios in os.listdir("/content/drive/MyDrive/dataset"):
        shutil.copy(f"/content/drive/MyDrive/dataset/{audios}", "/content/Vietnamese_RVC/dataset")
elif not any(f.lower().endswith(("wav", "mp3", "flac", "ogg", "opus", "m4a", "mp4", "aac", "alac", "wma", "aiff", "webm", "ac3")) for f in os.listdir("/content/Vietnamese_RVC/dataset") if os.path.isfile(os.path.join("/content/Vietnamese_RVC/dataset", f))):
    uploaded = files.upload()
    for f in list(uploaded.keys()):
        input_path = os.path.join("dataset", os.path.basename(f).replace(' ', '_').replace('(', '').replace(')', '').replace('{', '').replace('}', ''))
        shutil.move(f, input_path)

if not ten_mo_hinh: raise ValueError("Vui lòng cung cấp tên để tiến hành!")

preprocess(ten_mo_hinh, toc_do_lay_mau, 2, True, True, "dataset", False, 0.7)
extract(ten_mo_hinh, "v2", phuong_phap_trich_xuat, True, hop_length, 2, 0, toc_do_lay_mau, "contentvec_base", None)
create_index(ten_mo_hinh, "v2", "Auto")

clear_output()
display(Button(description="\u2714 Hoàn Tất!", button_style="success"))

In [ ]:
#@title 🤖 **Huấn luyện mô hình**
import os
import codecs

from ipywidgets import Button
from IPython.display import clear_output

%cd /content/Vietnamese_RVC

from main.tools import huggingface

def training(model_name, rvc_version, save_every_epoch, save_only_latest, save_every_weights, total_epoch, sample_rate, batch_size, gpu, pitch_guidance, not_pretrain, custom_pretrained, pretrain_g, pretrain_d, detector, threshold, clean_up, cache, model_author, vocoder, checkpointing):
    sr = int(float(sample_rate.rstrip("k")) * 1000)
    if not model_name: raise ValueError("Vui lòng cung cấp tên để tiến hành huấn luyện!")

    model_dir = os.path.join("assets", "logs", model_name)
    if not any(os.path.isfile(os.path.join(model_dir, f"{rvc_version}_extracted", f)) for f in os.listdir(os.path.join(model_dir, f"{rvc_version}_extracted"))): raise FileNotFoundError("Không tìm thấy dữ liệu đã được xử lý!")

    if not not_pretrain:
        if not custom_pretrained:
            pretrained_selector = {True: {32000: ("f0G32k.pth", "f0D32k.pth"), 40000: ("f0G40k.pth", "f0D40k.pth"), 44100: ("f0G44k.pth", "f0D44k.pth"), 48000: ("f0G48k.pth", "f0D48k.pth")}, False: {32000: ("G32k.pth", "D32k.pth"), 40000: ("G40k.pth", "D40k.pth"), 44100: ("G44k.pth", "D44k.pth"), 48000: ("G48k.pth", "D48k.pth")}}
            pg, pd = pretrained_selector[pitch_guidance][sr]
        else:
            if not pretrain_g: raise FileNotFoundError("Vui lòng cung cấp huấn luyện trước G")
            if not pretrain_d: raise FileNotFoundError("Vui lòng cung cấp huấn luyện trước D")

            pg, pd = pretrain_g, pretrain_d

        pretrained_G, pretrained_D = (os.path.join("assets", "models", f"pretrained_{rvc_version}", f"{vocoder if vocoder != 'Default' else ''}{pg}"), os.path.join("assets", "models", f"pretrained_{rvc_version}", f"{vocoder if vocoder != 'Default' else ''}{pd}")) if not custom_pretrained else (os.path.join("assets", "models", f"pretrained_custom", pg), os.path.join("assets", "models", f"pretrained_custom", pd))
        download_version = codecs.decode(f"uggcf://uhttvatsnpr.pb/NauC/Ivrganzrfr-EIP-Cebwrpg/erfbyir/znva/cergenvarq_i{'2' if rvc_version == 'v2' else '1'}/", "rot13")

        if not custom_pretrained:
            try:
                if not os.path.exists(pretrained_G): huggingface.HF_download_file(f"{download_version}{pg}", os.path.join("assets", "models", f"pretrained_{rvc_version}", f"{vocoder if vocoder != 'Default' else ''}{pg}"))
                if not os.path.exists(pretrained_D): huggingface.HF_download_file(f"{download_version}{pd}", os.path.join("assets", "models", f"pretrained_{rvc_version}", f"{vocoder if vocoder != 'Default' else ''}{pd}"))
            except:
                print("Sẽ không sử dụng huấn luyện trước vì thất bại trong việc cài đặt chúng!")
                pretrained_G, pretrained_D = None, None
        else:
            if not os.path.exists(pretrained_G): raise FileNotFoundError("Không tìm thấy huấn luyện trước G")
            if not os.path.exists(pretrained_D): raise FileNotFoundError("Không tìm thấy huấn luyện trước D")
    else: print("Sẽ không sử dụng huấn luyện trước!")

    print("Bắt đầu huấn luyện...")

    !python main/inference/train.py --model_name $model_name --rvc_version $rvc_version --save_every_epoch $save_every_epoch --save_only_latest $save_only_latest --save_every_weights $save_every_weights --total_epoch $total_epoch --sample_rate $sr --batch_size $batch_size --gpu $gpu --pitch_guidance $pitch_guidance --overtraining_detector $detector --overtraining_threshold $threshold --cleanup $clean_up --cache_data_in_gpu $cache --g_pretrained_path $pretrained_G --d_pretrained_path $pretrained_D --model_author $model_author --vocoder $vocoder --checkpointing $checkpointing

#@markdown **Tên của mô hình và tốc độ lấy mẫu để tiến hành huấn luyện mô hình**
ten_mo_hinh = "Anh" #@param {"type":"string", placeholder: "Tên mô hình"}
toc_do_lay_mau = "48k" #@param ["32k", "40k", "44.1k", "48k"]
#@markdown **Số lượng kỷ nguyên của mô hình và bao nhiêu kỷ nguyên sẽ lưu một lần**
so_luong_ky_nguyen = 300 #@param {"type":"slider", min:1, max:10000}
tan_suat_luu = 50 #@param {"type":"slider", min:1, max:10000}
#@markdown **Tùy chọn về kích thước lô và có sử dụng bộ nhớ đệm hay không**
kich_thuoc_lo = 8 #@param {"type":"slider", min:1, max:20}
bo_nho_dem = False #@param {"type":"boolean"}
#@markdown **Tùy chọn về tự động kiểm tra huấn luyện quá sức và sử dụng tùy chỉnh mô hình huấn luyện trước**
kiem_tra_huan_luyen = False #@param {"type":"boolean"}
tuy_chinh_huan_luyen_truoc = False #@param {"type":"boolean"}

if tuy_chinh_huan_luyen_truoc:
    model_name = os.listdir(os.path.join("assets", "models", "pretrained_custom"))

    if len(model_name) >= 2:
        for m in model_name:
            print(f"{model_name.index(m)}. {m}")

        while 1:
            try:
                model_index_d = int(input("Nhập số thứ tự của mô hình D: "))
                if 0 <= model_index_d < len(model_name):
                    selected_model_D = model_name[model_index_d]
                    print(f"Bạn đã chọn mô hình D: {selected_model_D}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        while 1:
            try:
                model_index_g = int(input("Nhập số thứ tự của mô hình G: "))
                if 0 <= model_index_g < len(model_name):
                    selected_model_G = model_name[model_index_g]
                    print(f"Bạn đã chọn mô hình G: {selected_model_G}")
                    break
                else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
            except ValueError:
                print("Vui lòng nhập một số nguyên.")
            except IndexError:
                print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

        if selected_model_D == selected_model_G: raise ValueError("Mô hình D và G giống nhau!")
        clear_output()
    else: print("không tìm thấy mô hình huấn luyện trước nào!")

training(ten_mo_hinh, "v2", tan_suat_luu, True, True, so_luong_ky_nguyen, toc_do_lay_mau, kich_thuoc_lo, 0, True, False, tuy_chinh_huan_luyen_truoc, selected_model_G, selected_model_D, kiem_tra_huan_luyen, 50, False, bo_nho_dem, None, "Default", False)
Button(description="\u2714 Hoàn tất!", button_style="success")

In [ ]:
#@title 📦 **Nén và lưu mô hình** 📦

#@markdown **Chạy ô này và chọn tệp mô hình cần nén!**

import os
import zipfile

from ipywidgets import Button
from IPython.display import clear_output

def get_index(model):
    return next((f for f in [os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index") and "trained" not in name] if model.split(".")[0] in f), "")

model_name = sorted(list(model for model in os.listdir(os.path.join("assets", "weights")) if model.endswith(".pth") and not model.startswith("G_") and not model.startswith("D_")))
indexpath = sorted([os.path.join(root, name) for root, _, files in os.walk(os.path.join("assets", "logs"), topdown=False) for name in files if name.endswith(".index")])

if len(model_name) < 1: raise ValueError("Vui lòng cung cấp mô hình để tiến hành chuyển đổi!")
elif len(model_name) == 1:
    model_path = os.path.join("assets", "weights", model_name[0])
    index_path = get_index(os.path.basename(model_name[0])[0])
else:
    print("Tìm thấy nhiều hơn 1 mô hình, vui lòng nhập số thứ tự của mô hình để tiến hành chuyển đổi!")
    for m in model_name:
        print(f"{model_name.index(m)}. {m}")

    while 1:
        try:
            model_index = int(input("Nhập số thứ tự của mô hình: "))
            if 0 <= model_index < len(model_name):
                selected_model = model_name[model_index]
                print(f"Bạn đã chọn mô hình: {selected_model}")
                break
            else: print("Số thứ tự không hợp lệ. Vui lòng nhập lại.")
        except ValueError:
            print("Vui lòng nhập một số nguyên.")
        except IndexError:
            print("Số thứ tự vượt quá phạm vi. Vui lòng nhập lại.")

    model_path = os.path.join("assets", "weights", selected_model)
    index_path = get_index(os.path.basename(selected_model).split("_")[0])

if not index_path: print("Không tìm thấy chỉ mục!")

zip_file_path = os.path.join("assets", os.path.basename(model_path).split("_")[0] + ".zip")

with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    zipf.write(model_path, os.path.basename(model_path))
    if index_path: zipf.write(index_path, os.path.basename(index_path))

clear_output()
print(f"Đường dẫn mô hình của bạn: {zip_file_path}")
Button(description="\u2714 Hoàn tất!", button_style="success")